In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime


In [ ]:
file_csv = '../raw_data/final_df.csv'
df_raw_data = pd.read_csv(file_csv)


In [ ]:
df.drop(columns=['run_1_raw_post_race_rating_int', 'run_1_raw_post_race_rating_symbol', 'run_1_final_rating_int',
            'run_1_race_type', 'run_1_race_class_normalised', 'run_1_race_class', 'run_1_track_type', 'run_1_win_lose',
            'run_1_dsr', 'run_2_raw_post_race_rating_int', 'run_2_raw_post_race_rating_symbol', 'run_2_final_rating_int',
            'run_2_race_type', 'run_2_race_class', 'run_2_race_class_normalised', 'run_2_track_type', 'run_2_win_lose',
            'run_2_dsr', 'run_3_raw_post_race_rating_int', 'run_3_raw_post_race_rating_symbol', 'run_3_final_rating_int',
            'run_3_race_type', 'run_3_race_class', 'run_3_race_class_normalised', 'run_3_track_type', 'run_3_win_lose',
            'run_3_dsr', 'run_4_raw_post_race_rating_int', 'run_4_raw_post_race_rating_symbol', 'run_4_final_rating_int',
            'run_4_race_type', 'run_4_race_class', 'run_4_race_class_normalised', 'run_4_track_type', 'run_4_win_lose',
            'run_4_dsr', 'run_5_raw_post_race_rating_int', 'run_5_raw_post_race_rating_symbol', 'run_5_final_rating_int',
            'run_5_race_type', 'run_5_race_class', 'run_5_race_class_normalised', 'run_5_track_type', 'run_5_win_lose',
            'run_5_dsr', 'run_6_raw_post_race_rating_int', 'run_6_raw_post_race_rating_symbol', 'run_6_final_rating_int',
            'run_6_race_type', 'run_6_race_class', 'run_6_race_class_normalised', 'run_6_track_type', 'run_6_win_lose',
            'run_6_dsr', 'run_7_raw_post_race_rating_int', 'run_7_raw_post_race_rating_symbol', 'run_7_final_rating_int',
            'run_7_race_type', 'run_7_race_class', 'run_7_race_class_normalised', 'run_7_track_type', 'run_7_win_lose',
            'run_7_dsr', 'run_8_raw_post_race_rating_int', 'run_8_raw_post_race_rating_symbol', 'run_8_final_rating_int',
            'run_8_race_type', 'run_8_race_class', 'run_8_race_class_normalised', 'run_8_track_type', 'run_8_win_lose',
            'run_8_dsr', 'meeting_name', 'country_code', 'distance_unit','distance_furlongs', 'prize_money_currency',
            'jockey_allowance_unit', 'handicap_weight_unit', 'jockey_name', 'trainer_name',
            'pre_race_master_rating_symbol', 'post_race_master_rating_symbol', 'post_race_master_rating_int',
            'bet365_odds', 'pmu_odds', 'meeting_id', 'distance_raw_furlongs', 'number', 'horse_id',  'dam', 'sire',
            'Date', 'id_lewagon'], inplace=True)
df['gear'] = df['gear'].apply(lambda x: 0 if pd.isna(x) else 1)
df['rating_oficial'] = df['OffR'].fillna(df['official rating'])
df['rating_oficial'] = df['official rating'].fillna(df['OffR'])
df['finish_position'].fillna(df['Place'], inplace=True)

df = df[df['barrier'] <= 20]
df['failed_to_finish_reason'] = df['failed_to_finish_reason'].apply(lambda x: 0 if pd.isna(x) else 1)
df['margin'] = df.apply(lambda row: row['distance'] if pd.isna(row['margin']) and (row['win_or_lose'] == 1 or row['failed_to_finish_reason'] == 1) else row['margin'], axis=1)
df['date'] = pd.to_datetime(df['date'])
df['birth_date'] = pd.to_datetime(df['birth_date'])
df['current_age'] = (((df['date'] - df['birth_date']).dt.days ) ).astype(float)

In [ ]:
df_sorted = df.sort_values(by=['horse_name', 'date'])
df_sorted[['date', 'horse_name', 'dslr']].isna().sum()

In [ ]:
df_sorted[['date', 'horse_name', 'dslr']].groupby('horse_name').agg({'date': 'diff'}).isna().sum()

In [ ]:
df_sorted['dslr'] = df_sorted['dslr'].fillna(df_sorted.groupby('horse_name')['date'].diff().dt.days)

In [ ]:
df_sorted[['date', 'horse_name', 'dslr', 'current_age', 'age']][df_sorted.dslr.isna()].age.value_counts()

In [ ]:
import pipeline_cleaning


In [ ]:
clean_data = pipeline_cleaning.clean_data(df_raw_data)
clean_data

In [ ]:
clean_data['date'] = pd.to_datetime(clean_data['date'])
clean_data.drop(columns=['tainer_id', 'margin', 'dslr','rating_oficial',
                    'last_traded_price', 'finish_position', 'event_number',
                    'pre_race_master_rating_int',
                    'post_time'], axis=1, inplace=True)# for now
clean_data.dropna(inplace=True) #instead of imputer for now

In [ ]:
clean_data.isna().sum()

In [ ]:
df_grouped = clean_data.groupby(by=['jockey_id']).agg({'win_or_lose': 'sum'}).sort_values(by='win_or_lose', ascending=False)
df_grouped

In [ ]:
quantiles = df_grouped['win_or_lose'].quantile([0.2, 0.4, 0.6, 0.8])

# Classify jockey_id into 5 groups based on the number of wins
def classify_group(win_or_lose):
    if win_or_lose <= quantiles[0.2]:
        return 1
    elif win_or_lose <= quantiles[0.4]:
        return 2
    elif win_or_lose <= quantiles[0.6]:
        return 3
    elif win_or_lose <= quantiles[0.8]:
        return 4
    else:
        return 5

df_grouped['win_or_lose_class'] = df_grouped['win_or_lose'].apply(classify_group)
df_grouped

In [ ]:
len(clean_data)

In [ ]:
clean_data.merge(df_grouped, how='left', left_on='jockey_id', right_on='jockey_id')
clean_data

In [ ]:
colunas = ['15_mins', '10_mins', '5_mins', '3_mins', '2_mins', '1_min_']

df_sem_nan = clean_data.dropna(subset=colunas, how='all')


In [ ]:
len(df_sem_nan)

In [ ]:
media_valores = df_sem_nan[colunas].mean(axis=1)

for coluna in colunas:
    df_sem_nan[coluna].fillna(media_valores, inplace=True)

In [ ]:
df_sem_nan.isna().sum()

In [ ]:
number_of_nas = df_sem_nan[['15_mins', '10_mins', '5_mins', '3_mins', '2_mins', '1_min_']].isna().sum().sum()
while number_of_nas > 0:
    df_sem_nan['1_min_'] = df_sem_nan['1_min_'].fillna(df_sem_nan['2_mins'])
    df_sem_nan['2_mins'] = df_sem_nan['2_mins'].fillna(df_sem_nan['3_mins'])
    df_sem_nan['3_mins'] = df_sem_nan['3_mins'].fillna(df_sem_nan['5_mins'])
    df_sem_nan['5_mins'] = df_sem_nan['5_mins'].fillna(df_sem_nan['10_mins'])
    df_sem_nan['10_mins'] = df_sem_nan['10_mins'].fillna(df_sem_nan['15_mins'])

    df_sem_nan['15_mins'] = df_sem_nan['15_mins'].fillna(df_sem_nan['10_mins'])
    df_sem_nan['10_mins'] = df_sem_nan['10_mins'].fillna(df_sem_nan['5_mins'])
    df_sem_nan['5_mins'] = df_sem_nan['5_mins'].fillna(df_sem_nan['3_mins'])
    df_sem_nan['3_mins'] = df_sem_nan['3_mins'].fillna(df_sem_nan['2_mins'])
    df_sem_nan['2_mins'] = df_sem_nan['2_mins'].fillna(df_sem_nan['1_min_'])
    number_of_nas = df_sem_nan[['15_mins', '10_mins', '5_mins', '3_mins', '2_mins', '1_min_']].isna().sum().sum()

In [ ]:
df_sem_nan.shape

In [ ]:
df_sem_nan.to_csv('../raw_data/withoutna.csv')

In [ ]:
jockey_class = pipeline_cleaning.transforming_data(clean_data, jockey_id=True)[0]


In [ ]:
jockey_class.columns

In [ ]:
clean_data['date'] = pd.to_datetime(clean_data['date'])
clean_data.drop(columns=['jockey_id', 'tainer_id', 'margin', 'finish_position', 'event_number'], axis=1, inplace=True)
clean_data.dropna(inplace=True) #instead of imputer
df_train = clean_data[(clean_data['date'].dt.year != 2022) & (clean_data['date'].dt.year != 2023)]
df_val = clean_data[clean_data['date'].dt.year == 2022]
df_test = clean_data[clean_data['date'].dt.year == 2023]
df_train.drop(columns=['date'], axis=1, inplace=True)
df_val.drop(columns=['date'], axis=1, inplace=True)
df_test.drop(columns=['date'], axis=1, inplace=True)

categorical_col = ['barrier', 'track_condition', 'race_type', 'track_type',
                    'race_class_normalised', 'race_class']
num_col = ['distance', 'total_prize_money', 'jockey_allowance',
            'handicap_weight', 'dslr', 'official rating', 'wfa',
            'weight_adjustment', 'betfair_starting_price',
            'pre_race_master_rating_int', 'starting_price', 'current_age']


In [ ]:
[col for col in df_train.columns if col not in (num_col+categorical_col)]

In [ ]:
df_test

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [ ]:
categorical_preprocessor = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])
numerical_preprocessor = Pipeline([
    ('scaler', StandardScaler())
])
pipeline = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_col),
    ('numerical', numerical_preprocessor, num_col)
], remainder="passthrough", sparse_threshold=0)
pipeline

In [ ]:
pipeline.fit(df_train)
df_train_transformed = pipeline.transform(df_train)
df_val_transformed = pipeline.transform(df_val)
df_test_transformed = pipeline.transform(df_test)

In [ ]:
pipeline.transform(df_train)

In [ ]:
# pipeline.get_feature_names_out()


In [ ]:
categorical_feature_names = pipeline.named_transformers_['categorical'].named_steps['onehot'].get_feature_names_out(input_features=categorical_col)

# Obter os nomes das colunas numéricas
numerical_feature_names = num_col
remainder_col_names = [col for col in df_train.columns if col not in (num_col+categorical_col)]

# Combinar os nomes das colunas categóricas e numéricas
all_feature_names = list(categorical_feature_names) + numerical_feature_names + remainder_col_names
all_feature_names

In [ ]:
df_test_transformed_with_columns = pd.DataFrame(df_test_transformed, columns=all_feature_names)
df_val_transformed_with_columns = pd.DataFrame(df_val_transformed, columns=all_feature_names)
df_train_transformed_with_columns = pd.DataFrame(df_train_transformed, columns=all_feature_names)

In [ ]:
df_train_transformed_with_columns